# Creating XGBoost model. 

This is based on how we trained 55 separate models for 55 separate wavelengths. 

Note that the difference, other than using XGBoost model, is that Decision Trees does **not** require normalizing data. Hence we can go as it is. 

And since our data is pre-cleaned, we also do not need to put a pipeline into it. 

It is a plus that Decision Trees (and hence XGBoost) works best when the features are a collection of categorical and numerical features, OR purely numerical features, which the latter is for ours. 

And it's a plus if the number of features is far less than the number of training samples. We can drop features as well later during training randomly. 

However, since we are not familiar with XGBoost, and we have lots of features, tuning it is something of a requirement due to inexperience. 

In [2]:
storage_name = "baseline_xgboost_pred_1.txt"

PROJECT_ID = "sunlit-analyst-309609"
%env GCLOUD_PROJECT = $PROJECT_ID
%load_ext google.cloud.bigquery

env: GCLOUD_PROJECT=sunlit-analyst-309609


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tempfile

import numpy as np
import pandas as pd
from tqdm import tqdm

import xgboost as xgb 
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

import tensorflow as tf 

from google.cloud import bigquery
LOCATION = "us"

The Bayesian Optimization technique had been taken from https://www.kdnuggets.com/2019/07/xgboost-random-forest-bayesian-optimisation.html

In [ ]:
def bayesian_optimization(dataset, function, parameters):
    